In [2]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import load_model
from nltk.stem import SnowballStemmer

In [3]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('mbi_questions_frensh.json', 'r', encoding="utf8").read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model_fr.keras', compile=False)

In [4]:
def clean_up_sentence(sentence):
    # Convert to lowercase and tokenize the sentence
    sentence_words = nltk.word_tokenize(sentence.lower())
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    # Generate a bag of words for the sentence after cleaning and lowercasing
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    # Use the bag of words model to predict the class with lowercase input
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # Sort results by probability in descending order
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    # Get the response for the top intent
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [6]:
print("GO! Le chatbot se commence!")

while True:
    message = input("")
    
    if message.lower() == "merci, au revoir":
        print("Merci et bonne journée")
        break
    
    ints = predict_class(message)
    if ints:  # If an intent is detected
        res = get_response(ints, intents)
    else:
        res = "J'ai pas compris la question, merci de la poser autrement."
    print(res)

GO! Le chatbot se commence!
bonjour
1/1 [==============================] - 0s 31ms/step
Bonjour ! Comment puis-je vous aider aujourd'hui ?
Qu'est ce que MBI ?
1/1 [==============================] - 0s 26ms/step
Le réseau MBI est une agence de développement spécialisée basée à Patti, en Italie. Elle est présente dans 6 pays (Royaume-Uni, Italie, France, Grèce, Roumanie et Maroc) et propose une large gamme de services. Ses domaines d'expertise incluent le marketing digital, la conception de sites web, l'optimisation SEO ainsi que le développement web et mobile.
Quels avantages supplémentaires le réseau MBI offre-t-il aux clients?
1/1 [==============================] - 0s 29ms/step
Le réseau MBI propose des solutions de marketing digital et de SEO pour votre site web. Votre site bénéficiera d'un design moderne et réactif, adapté à tous les écrans. Nous proposons également des solutions e-commerce pour simplifier les processus, automatiser les flux de travail et intégrer les paiements.
mer

In [7]:
res

"Le réseau MBI propose des solutions de marketing digital et de SEO pour votre site web. Votre site bénéficiera d'un design moderne et réactif, adapté à tous les écrans. Nous proposons également des solutions e-commerce pour simplifier les processus, automatiser les flux de travail et intégrer les paiements."